# SQL for Data Scientist wnb
To be able to query data smoothly is a mandatory. You cannot just ask your friend to do it for you. You gotta do it your self. So here's I collect compilation about sql related posts. You have to practice it

- http://blog.yhat.com/posts/SQL-interview-questions.html
- http://blog.yhat.com/posts/summarizing-data-in-SQL.html

I use Pandas all the time and I never get exposed properly with SQL world. It's unpleasant situation since at every data science interview you will be asked about your SQL skill regardless whether you will use it in day to day work  or not (but mostly you will since most of the time the data is very big, so gotta do some aggretation in place before you can move it your own laptop to work with). Some times the interviewer doesn't even know about Pandas and even didn't know it exist. If you only tell them you can use Python but not SQL, they conclude that you doesn't know the logic behind data munging even tho you are excel at it with Pandas. True story man, so better prepare. Here's something in Panda's docs that will help you learning SQL if you're in the same situation like me https://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html. The guide is actually better for people who want to learn more about Pandas after they can do SQL but it can be also the other way around. 

There are only 8 important things in data munging SQL according to the docs

1. SELECT
- WHERE
- GROUP BY & AGGREGATE
- JOIN
 - INNER JOIN
 - LEFT OUTER JOIN
 - RIGHT JOIN
 - FULL JOIN
- UNION
- Analytic and aggregate functions
 - Top N rows with offset
 - Top N rows per group
- UPDATE
- DELETE

Where's the pivot?

If you have done some work with Pandas, this tutorial is for you. In order to illustrate the SQL in this tutorial, I need help from Python library called sqlalchemy. You can find how to install sqlalchemy yourself and learn many more about it. In this tutorial tho, I will only use it to simulate a real database engine and do visualize the result of query in table.

In [118]:
import pandas as pd
from sqlalchemy import create_engine

In [119]:
df_p = pd.read_csv('./data/tips.csv')
df_p.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [120]:
disk_engine = create_engine('sqlite:///./data/tips.db')
df_p.to_sql('tips', disk_engine, if_exists='replace', index=False)

In [121]:
df_sql = pd.read_sql_query('SELECT * FROM tips', disk_engine)
df_sql.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


# 1. SELECT

In Pandas, we usualy selecting column this way:

In [122]:
df_p[['total_bill', 'tip', 'smoker', 'time']].head()

,total_bill,tip,smoker,time
0,16.99,1.01,No,Dinner
1,10.34,1.66,No,Dinner
2,21.01,3.50,No,Dinner
3,23.68,3.31,No,Dinner
4,24.59,3.61,No,Dinner


In SQL that would be equivalent to:

```sql
SELECT total_bill, tip, smoker, time 
FROM tips 
LIMIT 5
```

In [123]:
query_select = """
    SELECT total_bill, tip, smoker, time
    FROM tips
    LIMIT 5
"""

pd.read_sql_query(query_select, disk_engine)

,total_bill,tip,smoker,time
0,16.99,1.01,No,Dinner
1,10.34,1.66,No,Dinner
2,21.01,3.50,No,Dinner
3,23.68,3.31,No,Dinner
4,24.59,3.61,No,Dinner


# 2. WHERE

In Pandas, I usually filter the rows by the column value this way:

In [124]:
df_p[df_p['time'] == 'Dinner'].head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


Where we basically passing the Series of True/False values into the dataframe and returning all rows with True

In SQL that filtering would be equivalent to:

In [125]:
query_where = """
    SELECT *
    FROM tips
    WHERE time = 'Dinner'
    LIMIT 5
"""
pd.read_sql_query(query_where, disk_engine)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In Pandas filtering with multiple condition can be done like this

In [126]:
df_p[(df_p['time'] == 'Dinner') & (df_p['tip'] > 5.00)].head()

,total_bill,tip,sex,smoker,day,time,size
23,39.42,7.58,Male,No,Sat,Dinner,4
44,30.40,5.60,Male,No,Sun,Dinner,4
47,32.40,6.00,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
59,48.27,6.73,Male,No,Sat,Dinner,4


That & logic in SQL would be like this

In [127]:
query = """
    SELECT *
    FROM tips
    WHERE time = 'Dinner' AND tip > 5.00
    LIMIT 5
"""

pd.read_sql_query(query, disk_engine)

,total_bill,tip,sex,smoker,day,time,size
0,39.42,7.58,Male,No,Sat,Dinner,4
1,30.40,5.60,Male,No,Sun,Dinner,4
2,32.40,6.00,Male,No,Sun,Dinner,4
3,34.81,5.20,Female,No,Sun,Dinner,4
4,48.27,6.73,Male,No,Sat,Dinner,4


The or in Pandas | also behave the same way with SQL equivalent to OR

In Pandas we can take the null this way

In [128]:
import numpy as np
frame = pd.DataFrame({'col1':['A', 'B', np.nan, 'C', 'D'], 
                      'col2':['F', np.nan, 'G', 'H', 'I']})

disk_engine_frame = create_engine('sqlite:///./data/frame.db')
frame.to_sql('frame', disk_engine_frame, if_exists='replace', index=False)

frame

,col1,col2
0,A,F
1,B,NaN
2,NaN,G
3,C,H
4,D,I


In [129]:
frame[frame['col2'].isnull()]

,col1,col2
1,B,NaN


In [130]:
frame[frame['col2'].notnull()]

,col1,col2
0,A,F
2,NaN,G
3,C,H
4,D,I


In SQL that would be

In [131]:
query = """
    SELECT *
    FROM frame
    WHERE col2 IS NULL
"""

pd.read_sql_query(query, disk_engine_frame)

,col1,col2
0,B,None


In [132]:
query = """
    SELECT *
    FROM frame
    WHERE col2 IS NOT NULL
"""

pd.read_sql_query(query, disk_engine_frame)

,col1,col2
0,A,F
1,None,G
2,C,H
3,D,I


# 3. GROUP BY & AGGREGATE

We going to do a lot of this. Sometimes we only need to do aggregation without group by (but not the other way around, you should do something after grouping), sometimes we do groupby on one variable then do aggregation on the other columns, sometimes group by can be done several layer (by more than one column) then do aggregation on each group or maybe several sequece like this: groupby -> aggregation -> groupby -> aggregation, or maybe both.  Sometimes even a very simple logic for human can be pretty complex and tricky to code the query. So put some high respect to query worker in you office whenever you can cause they did awesome job. There're several aggregation function that are built-in for both pandas and sql. For sql, it can also be tricky since different database engine do support some function but some don't. So I recommend you to learn deeply only on some very basics function. Some aggregation function that frequently needed are shown in the table below

function|pandas|sql
--|--|--
average|`mean()`|`avg()`
count|`count()`|`count()`
max|`max()`|`max()`
min|`min()`|`min()`
sum|`sum()`|`sum()`

One of the simplest situation is doing aggregation without groupby

I sometimes did this in Python

In [133]:
df_p['total_bill'].mean()

19.785942622950824

Or this

In [134]:
df_p['total_bill'].sum()

4827.770000000001

In SQL that would be like this

In [135]:
query = """
SELECT AVG(total_bill)
FROM tips
"""

pd.read_sql_query(query, disk_engine)

,AVG(total_bill)
0,19.785943


In [136]:
query = """
SELECT SUM(total_bill)
FROM tips
"""

pd.read_sql_query(query, disk_engine)

,SUM(total_bill)
0,4827.77


The second one is do groupby on one column, and then aggregation on some other column. For example, we going to group the data by the day, and then find the mean or average of the tip for each group (since we group by the day so it will be the average tip for each different day)

In [141]:
df_p.groupby('day')['tip'].mean()

day
Fri     2.734737
Sat     2.993103
Sun     3.255132
Thur    2.771452
Name: tip, dtype: float64

Or this way

In [143]:
df_p.groupby('day').agg({'tip':'mean'})

,tip
day,
Fri,2.734737
Sat,2.993103
Sun,3.255132
Thur,2.771452


In [144]:
query = """
SELECT day, AVG(tip)
FROM tips
GROUP BY day
"""

pd.read_sql_query(query, disk_engine)

,day,AVG(tip)
0,Fri,2.734737
1,Sat,2.993103
2,Sun,3.255132
3,Thur,2.771452


Multiple functions can also be applied at once for different column. This is aggregation things. For example with Pandas

In [18]:
df_p.groupby('day').agg({'tip':np.mean, 'day':'count'})

,tip,day
day,,
Fri,2.734737,19
Sat,2.993103,87
Sun,3.255132,76
Thur,2.771452,62


In SQL that would be like this

In [19]:
query = """
SELECT day, AVG(tip), COUNT(*)
FROM tips
GROUP BY day
"""
pd.read_sql_query(query, disk_engine)

,day,AVG(tip),COUNT(*)
0,Fri,2.734737,19
1,Sat,2.993103,87
2,Sun,3.255132,76
3,Thur,2.771452,62


That `COUNT(*)` can also be replaced by `COUNT(day)`. The `COUNT(*)` there means counting the row of each group resulting from the `GROUP BY`, since we group it by `day` then it would be counting the number data on each day.

In Pandas, we can group by more than one column like this. The resulting DataFrame has some kind of hierarchical indexing.

In [20]:
df_p.groupby(['smoker', 'day']).agg({'day':np.size,'tip':np.mean})

tip  day
smoker day                
No     Fri   2.812500    4
       Sat   3.102889   45
       Sun   3.167895   57
       Thur  2.673778   45
Yes    Fri   2.714000   15
       Sat   2.875476   42
       Sun   3.516842   19
       Thur  3.030000   17

Or

In [21]:
df_p.groupby(['smoker', 'day']).agg({'tip':[np.mean, np.size]})

tip      
                 mean  size
smoker day                 
No     Fri   2.812500   4.0
       Sat   3.102889  45.0
       Sun   3.167895  57.0
       Thur  2.673778  45.0
Yes    Fri   2.714000  15.0
       Sat   2.875476  42.0
       Sun   3.516842  19.0
       Thur  3.030000  17.0

In SQL that would be this way

In [22]:
query = """
SELECT smoker, day, COUNT(*), AVG(tip)
FROM tips
GROUP BY smoker, day
"""

pd.read_sql_query(query, disk_engine)

,smoker,day,COUNT(*),AVG(tip)
0,No,Fri,4,2.812500
1,No,Sat,45,3.102889
2,No,Sun,57,3.167895
3,No,Thur,45,2.673778
4,Yes,Fri,15,2.714000
5,Yes,Sat,42,2.875476
6,Yes,Sun,19,3.516842
7,Yes,Thur,17,3.030000


Some tricks with Pandas, if you want the resulting dataframe result of pandas look like the one resulted from sql, you can try `reset_index()`.

In [167]:
df_flat = df_p.groupby(['smoker', 'day']).agg({'tip':[np.mean, np.size]}).reset_index()
df_flat

smoker   day       tip      
                    mean  size
0     No   Fri  2.812500   4.0
1     No   Sat  3.102889  45.0
2     No   Sun  3.167895  57.0
3     No  Thur  2.673778  45.0
4    Yes   Fri  2.714000  15.0
5    Yes   Sat  2.875476  42.0
6    Yes   Sun  3.516842  19.0
7    Yes  Thur  3.030000  17.0

But that only flatten the index. The next is to flatten the column part, you can do this trick, since there is no straight forward way to do that https://stackoverflow.com/questions/14507794/python-pandas-how-to-flatten-a-hierarchical-index-in-columns

In [166]:
df_flat.columns = df_flat.columns.get_level_values(0)
df_flat

,smoker,day,mean,size
0,No,Fri,2.812500,4.0
1,No,Sat,3.102889,45.0
2,No,Sun,3.167895,57.0
3,No,Thur,2.673778,45.0
4,Yes,Fri,2.714000,15.0
5,Yes,Sat,2.875476,42.0
6,Yes,Sun,3.516842,19.0
7,Yes,Thur,3.030000,17.0


# 4. JOIN

I wonder how the * there are many kinds of join: LEFT, RIGHT, INNER, OUTER, FULL etc. Why we need them all? From what I know, join and merge is only useful to denormalize tables on the database. And that can be done very simple right? Well to be honest I never be so clear about this join things. I will also check this docs: https://pandas.pydata.org/pandas-docs/stable/merging.html

## PRELIMINARY EXAMPLE

Let's make examle dataframe first. Let's say I'am a teacher of both math and physics. I have two tables in my spreadsheet of students grades of both of that subject.

In [23]:
math = pd.DataFrame({'name':['A', 'B', 'C', 'D', 'E'],
                     'math_grade':[1.,2.,3.,4.,5.]})
physics = pd.DataFrame({'name':['A', 'B', 'C', 'D', 'E'],
                        'physics_grade':[2.,3.,4.,5.,np.nan]})


disk_engine = create_engine('sqlite:///./data/grades.db')
math.to_sql('math', disk_engine, if_exists='replace', index=False)
physics.to_sql('physics', disk_engine, if_exists='replace', index=False)

In [24]:
math

,math_grade,name
0,1.0,A
1,2.0,B
2,3.0,C
3,4.0,D
4,5.0,E


In [25]:
query = """
SELECT *
FROM math
"""
pd.read_sql_query(query, disk_engine)

,math_grade,name
0,1.0,A
1,2.0,B
2,3.0,C
3,4.0,D
4,5.0,E


In [26]:
physics

,name,physics_grade
0,A,2.0
1,B,3.0
2,C,4.0
3,D,5.0
4,E,NaN


In [27]:
query = """
SELECT *
FROM physics
"""
pd.read_sql_query(query, disk_engine)

,name,physics_grade
0,A,2.0
1,B,3.0
2,C,4.0
3,D,5.0
4,E,NaN


I want to merge both table so that I can see the students grade on both subject more clearly or maybe later I want to see the correlation of both subject or something. But let's examine more clearly on the pandas docs:

```python
pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True, 
         suffixes=('_x', '_y'), copy=True, indicator=False)
```

As you can see there are many inputs that can be passed. Here's I rewrite what the docs say to make it easier for you:
- `left`: A DataFrame object
- `right`: Another DataFrame object
- `on`: Column (names) to join on. Must be found in both the left and the right DataFrame objects. If not passed and `left_on` and `right_on` are False, the intersection of the columns in the DataFrames will be inffered to be the join keys.
- `left_on`: Columns from the left DataFrame to use as keys. Can either be column names or arrays with length equal to the length of the DataFrame
- `right_on`: Columns from the right DataFrame to use as keys. Can either be column names or arrays with length equal to the length of the DataFrame
- `left_index`: If true, use the index (row labels) from the left DataFrame as it's join key(s). In the case of DataFrame with multiIndex (hierarchical), the number of levels must match the number of join keys from the right DataFrame
- `right_index`: Same usage as `left_index` for the right DataFrame
- `how`: One of `left`, `right`, `innner`, `outer`. Default to `inner`. See below for more detailed description of each method.
- `sort`: Sort the result DataFrame by the join keys in lexicographical order. Defaults to `True`, setting to `False` will improve performance substantially in many cases.
- `suffixes`: A tupple of string to apply to overlapping columns. Defaults to ('_x', '_y')
- `copy`: Always copy data (default `True`) from the passed DataFrame objects, even when reindexing is not necessary. Cannot be avoided in many cases but may increase performance / memory usage. The cases where copying can be avoided are somewhat pathological but this option is provided nonetheless.
- `indicator`: ...


Merge is very related to relational databases like SQL database structure. There are several cases to consider which are very important to understand:
- __one-to-one__ joins
- __many-to-one__ joins
- __many-to-many__ joins

In [28]:
pd.merge(math, physics)

,math_grade,name,physics_grade
0,1.0,A,2.0
1,2.0,B,3.0
2,3.0,C,4.0
3,4.0,D,5.0
4,5.0,E,NaN


In [29]:
query = """
SELECT *
FROM math
INNER JOIN physics
ON math.name = physics.name
"""
pd.read_sql_query(query, disk_engine)

,math_grade,name,name,physics_grade
0,1.0,A,A,2.0
1,2.0,B,B,3.0
2,3.0,C,C,4.0
3,4.0,D,D,5.0
4,5.0,E,E,NaN


Well the name is doubled. This is actually happened in sql

In [30]:
pd.merge(math, physics, on='name')

,math_grade,name,physics_grade
0,1.0,A,2.0
1,2.0,B,3.0
2,3.0,C,4.0
3,4.0,D,5.0
4,5.0,E,NaN


In [31]:
pd.merge(math, physics, on='name', how='left')

,math_grade,name,physics_grade
0,1.0,A,2.0
1,2.0,B,3.0
2,3.0,C,4.0
3,4.0,D,5.0
4,5.0,E,NaN


In [37]:
query = """
SELECT *
FROM math
LEFT OUTER JOIN physics
ON math.name = physics.name
"""
pd.read_sql_query(query, disk_engine)

,math_grade,name,name,physics_grade
0,1.0,A,A,2.0
1,2.0,B,B,3.0
2,3.0,C,C,4.0
3,4.0,D,D,5.0
4,5.0,E,E,NaN


In [38]:
pd.merge(math, physics, on='name', how='right')

,math_grade,name,physics_grade
0,1.0,A,2.0
1,2.0,B,3.0
2,3.0,C,4.0
3,4.0,D,5.0
4,5.0,E,NaN


```python
query = """
SELECT *
FROM math
RIGHT OUTER JOIN physics
ON math.name = physics.name
"""
pd.read_sql_query(query, disk_engine)
```

Very unfortunate we got this:

`sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported`


In [40]:
pd.merge(math, physics, on='name', how='outer')

,math_grade,name,physics_grade
0,1.0,A,2.0
1,2.0,B,3.0
2,3.0,C,4.0
3,4.0,D,5.0
4,5.0,E,NaN


```python
query = """
SELECT *
FROM math
NATURAL OUTER JOIN physics
ON math.name = physics.name
"""
pd.read_sql_query(query, disk_engine)
```

Our example of math and physics grades is actually an example of one-to-one relationship. In all the experiment we've done above, the resulting DataFrame are the same even though we veried the `how` parameters. This happens since this is a __one-to-one__ relationship. We even don't have to use `on` since pandas merge can automatically looking for intersection of both dataframe columns.

The `how` parameters actually specify something. It determines which keys are to be included in the resulting table. Here's the equivalent for SQL so we can learn both at the same time:

Merge methode|SQL Join Name|Description
--|--|--
`left`|`LEFT OUTER JOIN`|Use keys from left frames only
`right`|`RIGHT OUTER JOIN`|Use keys from right framse only
`outer`|`FULL OUTER JOIN`|Use __union__ of keys from both frames
`inner`|`INNER JOIN`|Use __intersection__ of keys from both frames

The difference between those four is actually just how you going to include the keys for the final merged dataframe. I think, the usage of those four merge method is different from one another depends on the case. 

In [50]:
trans = pd.DataFrame({'date': [1,2,3,4,5],
                      'trans_id': ['t1','t2','t3','t4','t5'],
                      'item': ['A', 'B', 'C', 'A', 'A'],
                      'quantity': [1,1,2,1,1]})
trans = trans.set_index('date')

items = pd.DataFrame({'item':['A', 'B', 'C', 'D', 'E'],
                      'price':[10,20,30,40,50]})

disk_engine = create_engine('sqlite:///./data/transitem.db')
trans.to_sql('trans', disk_engine, if_exists='replace', index=False)
items.to_sql('items', disk_engine, if_exists='replace', index=False)

In [51]:
trans

,item,quantity,trans_id
date,,,
1,A,1,t1
2,B,1,t2
3,C,2,t3
4,A,1,t4
5,A,1,t5


In [52]:
items

,item,price
0,A,10
1,B,20
2,C,30
3,D,40
4,E,50


The the above data frames example, I choose a transaction data with transaction id and the name of the item (usually the name is also normalized). Our goal is to merge the two df so that we can calculate the total expense of each transaction. Let's examine the `how` parameters works in our `trans` and `items` dataFrame.

In [53]:
pd.merge(trans, items, on='item', how='left')

,item,quantity,trans_id,price
0,A,1,t1,10
1,B,1,t2,20
2,C,2,t3,30
3,A,1,t4,10
4,A,1,t5,10


In [62]:
query = """
SELECT *
FROM trans
LEFT OUTER JOIN items
ON trans.item = items.item
"""

pd.read_sql_query(query, disk_engine)

,item,quantity,trans_id,item,price
0,A,1,t1,A,10
1,B,1,t2,B,20
2,C,2,t3,C,30
3,A,1,t4,A,10
4,A,1,t5,A,10


The `left` use keys from the left frame only. In our case, the left is the `trans` df. So all the keys (in our case `item`) of the left (`trans`) df would be preserved in the final merged df. We also can see that the order is also preserved. Actually this merged df is exactly what we want the final df to be. 

In [58]:
pd.merge(trans, items, on='item', how='right')

,item,quantity,trans_id,price
0,A,1.0,t1,10
1,A,1.0,t4,10
2,A,1.0,t5,10
3,B,1.0,t2,20
4,C,2.0,t3,30
5,D,NaN,NaN,40
6,E,NaN,NaN,50


This `right` use keys from the right frame only. As you can see there is D and E preserved on the item with NaN values both on quantity and trans_id since there are no transaction includes these D and E item. We can also see that the order follows the right df keys.

In [59]:
pd.merge(trans, items, on='item', how='outer')

,item,quantity,trans_id,price
0,A,1.0,t1,10
1,A,1.0,t4,10
2,A,1.0,t5,10
3,B,1.0,t2,20
4,C,2.0,t3,30
5,D,NaN,NaN,40
6,E,NaN,NaN,50


The `outer` would include the union of the keys of both dataframe. Since the union of both would be A, B, C, D and E, so all would be considered in the final df. How about the order? Since we not prioritize the keys of both left or right, the order would be choose according to the `keys` itself

In [60]:
pd.merge(trans, items, on='item', how='inner')

,item,quantity,trans_id,price
0,A,1,t1,10
1,A,1,t4,10
2,A,1,t5,10
3,B,1,t2,20
4,C,2,t3,30


The last `inner` take the intersection of the keys. Since the intersection would be A, B and C, only those keys would be included. About the order, as before since we do not prioritize either left or right, the order would be the internal if the keys instead.

Very unfortunate, we cannot do right and outer join. But that's ok I think since most of the time to merge we only need LEFT OUTER JOIN or INNER JOIN to do the work.

# 5. UNION

In [64]:
city1 = pd.DataFrame({'city': ['Chicago', 'San Fransisco', 'New York City'], 
                    'rank': range(1,4)})
city2 = pd.DataFrame({'city': ['Chicago', 'Boston', 'Los Angeles'],
                      'rank': [1,4,5]})

disk_engine = create_engine('sqlite:///./data/city.db')
city1.to_sql('city1', disk_engine, if_exists='replace', index=False)
city2.to_sql('city2', disk_engine, if_exists='replace', index=False)

We can do something like this in Pandas:

In [66]:
pd.concat([city1, city2])

,city,rank
0,Chicago,1
1,San Fransisco,2
2,New York City,3
0,Chicago,1
1,Boston,4
2,Los Angeles,5


In [67]:
query = """
SELECT *
FROM city1
UNION ALL
SELECT *
FROM city2
"""

pd.read_sql_query(query, disk_engine)

,city,rank
0,Chicago,1
1,San Fransisco,2
2,New York City,3
3,Chicago,1
4,Boston,4
5,Los Angeles,5


To drop the duplicates in pandas we can do

In [68]:
pd.concat([city1, city2]).drop_duplicates()

,city,rank
0,Chicago,1
1,San Fransisco,2
2,New York City,3
1,Boston,4
2,Los Angeles,5


In SQL that would be equivalent to UNION without ALL

In [71]:
query = """
SELECT *
FROM city1
UNION
SELECT *
FROM city2
"""

pd.read_sql_query(query, disk_engine)

,city,rank
0,Boston,4
1,Chicago,1
2,Los Angeles,5
3,New York City,3
4,San Fransisco,2


# 6. Pandas equivalent for some SQL analytics and aggregate functions

## Top N rows with offset

We will use tips data, so let's refreshed it first

In [74]:
df_p.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [79]:
disk_engine = create_engine('sqlite:///./data/tips.db')

To find top N of values on certain column in Pandas we can do this to see top 5 to 15

In [86]:
df_p.nlargest(10+5, columns='tip').tail(10)

,total_bill,tip,sex,smoker,day,time,size
183,23.17,6.50,Male,Yes,Sun,Dinner,4
214,28.17,6.50,Female,Yes,Sat,Dinner,3
47,32.40,6.00,Male,No,Sun,Dinner,4
239,29.03,5.92,Male,No,Sat,Dinner,3
88,24.71,5.85,Male,No,Thur,Lunch,2
181,23.33,5.65,Male,Yes,Sun,Dinner,2
44,30.40,5.60,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
85,34.83,5.17,Female,No,Thur,Lunch,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4


In [87]:
query = """
SELECT * FROM tips
ORDER BY tip DESC
LIMIT 10 OFFSET 5;
"""
pd.read_sql_query(query, disk_engine)

,total_bill,tip,sex,smoker,day,time,size
0,23.17,6.50,Male,Yes,Sun,Dinner,4
1,28.17,6.50,Female,Yes,Sat,Dinner,3
2,32.40,6.00,Male,No,Sun,Dinner,4
3,29.03,5.92,Male,No,Sat,Dinner,3
4,24.71,5.85,Male,No,Thur,Lunch,2
5,23.33,5.65,Male,Yes,Sun,Dinner,2
6,30.40,5.60,Male,No,Sun,Dinner,4
7,34.81,5.20,Female,No,Sun,Dinner,4
8,34.83,5.17,Female,No,Thur,Lunch,4
9,25.89,5.16,Male,Yes,Sat,Dinner,4


## Top N rows per group

Yeay I've done this before. group by the day and take top total_bill of each day

In [92]:
df_p.groupby(['day'])['total_bill'].nlargest(2)

day      
Fri   95     40.17
      90     28.97
Sat   170    50.81
      212    48.33
Sun   156    48.17
      182    45.35
Thur  197    43.11
      142    41.19
Name: total_bill, dtype: float64

In [103]:
(df_p.assign(rn = df_p.sort_values(['total_bill'], ascending=False).groupby(['day']).cumcount() + 1)
                            .query('rn<3')
                            .sort_values(['day', 'rn'])
)

,total_bill,tip,sex,smoker,day,time,size,rn
95,40.17,4.73,Male,Yes,Fri,Dinner,4,1
90,28.97,3.00,Male,Yes,Fri,Dinner,2,2
170,50.81,10.00,Male,Yes,Sat,Dinner,3,1
212,48.33,9.00,Male,No,Sat,Dinner,4,2
156,48.17,5.00,Male,No,Sun,Dinner,6,1
182,45.35,3.50,Male,Yes,Sun,Dinner,3,2
197,43.11,5.00,Female,Yes,Thur,Lunch,4,1
142,41.19,5.00,Male,No,Thur,Lunch,5,2


It turns out quite complicated dude! Yes it turns out that I still miss this kind of skills that is actually very important in analytics. Even in Pandas!!! A simple query, sort or groupby might be easy, but a complex aggregation is not easy at all. Even it seems easy: return the whole dataframe of 2 top_total bill groupby day. Some database engine even doesn't support some function

# 7. UPDATE

# 8. DELETE

Better not learn those two if you don't know what you're doing